# Spotify API

<img width="1000" src = "https://www.markhneedham.com/blog//uploads/2020/01/spotify-logo.png" />

- Spotify é um serviço digital que dá acesso instantâneo a milhões de músicas, podcasts, vídeos e outros conteúdos de criadores no mundo todo.

- É liberado o acesso às suas APIs para que os desenvolvedores possam explorar seus dados.

- [Url para os endpoints do Spotify](https://developer.spotify.com/) 





## Libraries Import

In [1]:
import os
import json
import base64
import pandas as pd
import lyricsgenius as lg
from requests import post, get
from dotenv import load_dotenv

#### Load personal keys

In [2]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

genius = lg.Genius(os.getenv("GENIUS_TOKEN"), skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)

## Making requests to Spotify API

#### Function to get API's access token

In [3]:
def get_token():
    auth_string = CLIENT_ID + ":" + CLIENT_SECRET
    auth_string_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_string_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {"grant_type": "client_credentials"}
    response = post(url, headers=headers, data=data)
    json_response = json.loads(response.content)
    token = json_response.get("access_token")

    return token

In [4]:
def default_headers(token):
    return {"Authorization": "Bearer " + token}


In [5]:
token = get_token()

#### Search for an artist

In [6]:
def get_artist(token, artist):
    url = "https://api.spotify.com/v1/search"
    headers = default_headers(token)
    query = f"?q={artist}&type=artist&limit=1"

    url_with_params = url + query
    response = get(url_with_params, headers=headers)

    json_reponse = json.loads(response.content)["artists"]["items"]
    return json_reponse

In [7]:
artist_infos = get_artist(token, "Seu Jorge")
artist_infos

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0i1s9WcIu0PrUvHzALgofo'},
  'followers': {'href': None, 'total': 1634593},
  'genres': ['mpb', 'samba-rock'],
  'href': 'https://api.spotify.com/v1/artists/0i1s9WcIu0PrUvHzALgofo',
  'id': '0i1s9WcIu0PrUvHzALgofo',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5eb4e8989ba01fcdd07ca78f4f0',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab676161000051744e8989ba01fcdd07ca78f4f0',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f1784e8989ba01fcdd07ca78f4f0',
    'width': 160}],
  'name': 'Seu Jorge',
  'popularity': 71,
  'type': 'artist',
  'uri': 'spotify:artist:0i1s9WcIu0PrUvHzALgofo'}]

In [8]:
artist_id = artist_infos[0]["id"]
artist_id

'0i1s9WcIu0PrUvHzALgofo'

#### Get top tracks of selected artist

In [9]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = default_headers(token)

    response = get(url, headers=headers)
    json_response = json.loads(response.content)["tracks"]
    return json_response

In [10]:
songs = get_songs_by_artist(token, artist_id)
songs

[{'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
     'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
     'id': '0LyfQWJT6nXafLPZqxe9Of',
     'name': 'Various Artists',
     'type': 'artist',
     'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN'

In [11]:
songs_details = []
artist = genius.search_artist("Seu Jorge", max_songs=1)

for song in songs:
    song_lyrics = artist.song(song["name"]).lyrics

    songs_details.append({
        "name": song["name"],
        "artist": "Seu Jorge",
        "type": song["album"]["album_type"],
        "url": song["external_urls"]["spotify"],
        "popularity": song["popularity"],
        "duration (ms)": song["duration_ms"],
        "release_date": song["album"]["release_date"],
        "lyrics": song_lyrics
                })

Searching for songs by Seu Jorge...

Song 1: "Rebel Rebel"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "MTG QUEM NÃO QUER SOU EU" by Seu Jorge...
Done.
Searching for "Amiga da Minha Mulher" by Seu Jorge...
Done.
Searching for "Mina do Condomínio" by Seu Jorge...
Done.
Searching for "Burguesinha" by Seu Jorge...
Done.
Searching for "Final de Semana" by Seu Jorge...
Done.
Searching for "Felicidade" by Seu Jorge...
Done.
Searching for "Vou Festejar" by Seu Jorge...
Done.
Searching for "Quem Não Quer Sou Eu" by Seu Jorge...
Done.
Searching for "É Isso Aí (The Blower'S Daughter)" by Seu Jorge...
Done.
Searching for "Pra Melhorar" by Seu Jorge...
Done.


In [12]:
songs_details

[{'name': 'MTG QUEM NÃO QUER SOU EU',
  'artist': 'Seu Jorge',
  'type': 'single',
  'url': 'https://open.spotify.com/track/6RjSXi9cPGroetXeNsWFGL',
  'popularity': 78,
  'duration (ms)': 123692,
  'release_date': '2024-05-10',
  'lyrics': '3 ContributorsMTG QUEM NÃO QUER SOU EU Lyrics\n\nEu vou ficar a noite em claro sem pegar no sono\nMeditando sobre o que de fato aconteceu\nEu até pensei que fosse terminar na cama\nComo era de costume entre você e eu\nEu fiz de tudo, mas era tarde\nFoi o que eu podia dar, você não entendeu\nEu quis ir fundo e você com medo\nTirou onda, pois agora quem não quer sou eu (E aê, DJ Topo?)\n\nÉ, quem não quer sou eu (Maldita de ex, me ligando de madrugada)\nQuem não quer sou eu\nPois é (Maldita de ex, me ligando de madrugada)\nÉ, quem não quer sou eu (Maldita de ex, me ligando de madrugada)\nQuem não quer sou eu\nPois é (Maldita de ex, me ligando de madrugada)\n\nVai a noite, vem o dia, e eu aqui pensando\nUm cigarro atrás do outro e fumo sem parar\nDa ja

In [13]:
songs_df = pd.DataFrame(songs_details).sort_values(by=["popularity"], ascending=False).reset_index(drop=True)
songs_df

,name,artist,type,url,popularity,duration (ms),release_date,lyrics
0,MTG QUEM NÃO QUER SOU EU,Seu Jorge,single,https://open.spotify.com/track/6RjSXi9cPGroetX...,78,123692,2024-05-10,3 ContributorsMTG QUEM NÃO QUER SOU EU Lyrics\...
1,Amiga da Minha Mulher,Seu Jorge,album,https://open.spotify.com/track/6TVmNRrCRzU3Nly...,66,248080,2011-10-01,10 ContributorsAmiga Da Minha Mulher Lyrics\n\...
2,Mina do Condomínio,Seu Jorge,album,https://open.spotify.com/track/6Vfcu5p7cPcoVIm...,64,400626,2007-01-01,4 ContributorsMina do Condomínio Lyrics\n\nTô ...
3,Burguesinha,Seu Jorge,album,https://open.spotify.com/track/5iEEpVPlnH41wyA...,62,258186,2007-01-01,11 ContributorsBurguesinha Lyrics\n\nIh... olh...
4,Final de Semana,Seu Jorge,single,https://open.spotify.com/track/41sjmSYBlafAQrf...,62,201682,2021-02-12,8 ContributorsFinal de Semana Lyrics\n\nLiguei...
5,Felicidade,Seu Jorge,album,https://open.spotify.com/track/2bDc2CMq7MMAURw...,60,267413,2015-03-31,5 ContributorsFelicidade Lyrics\nFelicidade é ...
6,Quem Não Quer Sou Eu,Seu Jorge,album,https://open.spotify.com/track/7JFGnfONNe5vepb...,58,326493,2011-10-01,7 ContributorsQuem Não Quer Sou Eu Lyrics\n\nV...
7,É Isso Aí (The Blower'S Daughter),Seu Jorge,album,https://open.spotify.com/track/6tvnj77OJD0DJTg...,57,318026,2005-11-17,1 ContributorThe blowers daughter / e isso ai ...
8,Vou Festejar,Seu Jorge,compilation,https://open.spotify.com/track/4oNk9WpCEytgOdY...,54,199560,2016-07-22,3 ContributorsOgum Lyrics\n\nEu sou descendent...
9,Pra Melhorar,Seu Jorge,album,https://open.spotify.com/track/7oPB5rRWan5Seij...,51,256613,2022-05-18,6 ContributorsPra Melhorar Lyrics\nQuando você...


In [22]:
print(songs_df["lyrics"][6])

7 ContributorsQuem Não Quer Sou Eu Lyrics

Vou ficar a noite em claro sem pegar no sono
Meditando sobre o que de fato aconteceu
Eu até pensei que fosse terminar na cama
Como era de costume entre você e eu

Eu fiz de tudo mas era tarde
Foi o que eu podia dar você não entendeu
Eu quis ir fundo e você com medo
Tirou onda pois agora quem não quer sou eu
É... Quem não quer sou eu

Quem não quer sou eu
Pois é...

E vai a noite, vem o dia
E eu aqui pensando
Um cigarro atrás do outro
E eu fumo sem parar
Da janela eu vejo o trânsito congestionando
No meu peito o coração parece buzinar
You might also like
Eu fiz de tudo mas era tarde
Foi o que eu podia dar você não entendeu
Eu quis ir fundo e você com medo
Tirou onda pois agora quem não quer sou eu
É... Quem não quer sou eu

Quem não quer sou eu
Pois é...

Vou ficar a noite em claro sem pegar no sono
Meditando sobre o que de fato aconteceu
Eu até pensei que fosse terminar na cama
Como era de costume entre você e eu

Eu fiz de tudo mas era tarde
